## Task 1
### Solving ordinary differential equation of the second order by finite difference method

Equation:<br/>
$ -p(x) u''(x) + q(x) u'(x) + r(x) u(x) = f(x) $

x $ \in (a, b) $

Boundary values: <br/>
$ \alpha_{1}u(a) − \alpha_{2}u′(a) = \alpha, |\alpha_{1}| + |\alpha_{2}| \neq 0, \alpha_{1}\alpha_{2} ≥ 0, $ <br/>
$ \beta_{1}u(b) + \beta_{2}u′(b) = \beta, |\beta_{1}| + |\beta_{2}| \neq 0, \beta_{1}\beta_{2} ≥ 0. $


In [1]:
import numpy as np
from typing import Tuple, Callable
from math import e, sin, cos
from scipy.sparse import coo_matrix, vstack
import matplotlib.pyplot as plt

Function = Callable[[float], float]
Grid = Tuple[np.ndarray, float]
BoundaryValue = Tuple[float, float, float]

#### Solves linear algebraic systems with tridiagonal matrix
Uses [tridiagonal matrix algorighm](https://en.wikipedia.org/wiki/Tridiagonal_matrix_algorithm)


In [2]:
def solve_tridiagonal_system(matrix: np.ndarray, right_part: np.ndarray) -> np.ndarray:
    n = matrix.shape[0]
    alphas = [-1 * matrix[0][1] / matrix[0][0]]
    betas = [right_part[0] / matrix[0][0]]

    # The forward sweep
    for i in range(1, n):
        y = matrix[i, i] + matrix[i, i - 1] * alphas[i - 1]
        if i < n - 1:
            alphas.append(-1 * matrix[i, i + 1] / y)
        betas.append((right_part[i] - matrix[i, i - 1] * betas[i - 1]) / y)

    # Back substitution
    result = [betas[n - 1]]
    for i in range(n - 2, -1, -1):
        result.insert(0, alphas[i] * result[0] + betas[i])
    return np.array(result)

In [5]:
def get_true_error_norm(true_solution: np.ndarray, actual_solution: np.ndarray) -> float:
    return abs(true_solution - actual_solution).max()

In [6]:
def get_richardson_error_norm(current_solution: np.ndarray, next_solution: np.ndarray, r=2, p=2) -> float:
    n = len(current_solution)
    sum = 0
    [sum := sum + get_richardson_error(current_solution[i], next_solution[i * 2], r, p) for i in range(n)]
    return sum / n


def get_richardson_error(curent_solution, next_solution, r=2, p=2) -> float:
    return ((curent_solution - next_solution) / (r ** p - 1)) ** 2


def get_richardson_clarified_solution(current_solution, next_solution, r=2, p=2):
    n = len(next_solution)
    clarified_solution = np.zeros(n)
    errors = [0] * n
    for i in range(len(current_solution)):
        errors[i * 2] = get_richardson_error(current_solution[i], next_solution[i * 2], r, p)
    for i in range(len(current_solution) - 1):
        errors[i * 2 + 1] = (errors[i * 2] + errors[i * 2 + 2]) / 2
    errors[-1] = errors[-2]

    for i in range(n):
        clarified_solution[i] = next_solution[i] + errors[i]

    return clarified_solution

#### Transforms differential equation into an equal algebraic system

The method is based on replacing derivative with finite differences of $O(h^{2})$ order.
An approximated function is represented by values in points of the grid.

In [3]:
def solve_equation(
        p: Function,
        q: Function,
        r: Function,
        f: Function,
        grid: Grid,
        left_boundary_value: BoundaryValue,
        right_boundary_value: BoundaryValue
        ) -> np.ndarray:
    alpha_1, alpha_2, alpha = left_boundary_value
    beta_1, beta_2, beta = right_boundary_value

    points, step = grid
    n = len(points)

    matrix = np.zeros((n, n))
    right_part = np.zeros(n)

    # first row, left bounday values
    matrix[0][0] = alpha_1 + 3 / (2 * step) * alpha_2
    matrix[0][1] = -2 * alpha_2 / step
    matrix[0][2] = alpha_2 / (2 * step)
    right_part[0] = alpha

     ## Finite differences for inner points of the grid
    for index in range(1, n - 1):
        point = points[index]

        matrix[index][index - 1] = 1 / (step ** 2) - q(point) / (2 * step)
        matrix[index][index] = -2 / (step ** 2) - r(point)
        matrix[index][index + 1] = 1 / (step ** 2) + q(point) / (2 * step)
        right_part[index] = f(point)

    matrix[n - 1][n - 3] = beta_2 / (2 * step)
    matrix[n - 1][n - 2] = -2 * beta_2 / step
    matrix[n - 1][n - 1] = beta_1 + 3 * beta_2 / (2 * step)
    right_part[n - 1] = beta

    print("Matrix: \n", matrix)

    is_tridiagonal = alpha_2 == 0 and beta_2 == 0

    return np.linalg.solve(matrix, right_part)

In [133]:
u = lambda x: x ** 5 - 5 * x ** 4 + 5 * x ** 3 + 5 * x ** 2 - 6 * x
du = lambda x: 5 * x ** 4 - 20 * x ** 3 + 15 * x ** 2 + 10 * x - 6
ddu = lambda x: 10 * (2 * x ** 3 - 6 * x ** 2 + 3 * x + 1)

p = lambda x: 1
q = lambda x: 5
r = lambda x: -x
f = lambda x: ddu(x) + q(x) * du(x) - r(x) * u(x)

a = -1
b = 3

alpha_1 = 2
alpha_2 = 1
alpha = -24

left_boundary_value = (alpha_1, alpha_2, alpha)

beta_1 = 3
beta_2 = 1 / 8
beta = 3

right_boundary_value = (beta_1, beta_2, beta)

grid_size = 5
h = (b - a) / grid_size
grid = np.linspace(a - h / 2, b + h / 2, grid_size + 2, retstep=True)
result = solve_equation(p, q, r, f, grid, left_boundary_value, right_boundary_value)
print("Result: \n", result)

points, step = grid
true_solution = []
for point in points:
    true_solution.append(u(point))

print("True result: \n", true_solution)
true_error = get_true_error_norm(true_solution, result)

Matrix: 
 [[ 3.875    -2.5       0.625     0.        0.        0.        0.      ]
 [-1.5625   -3.725     4.6875    0.        0.        0.        0.      ]
 [ 0.       -1.5625   -2.925     4.6875    0.        0.        0.      ]
 [ 0.        0.       -1.5625   -2.125     4.6875    0.        0.      ]
 [ 0.        0.        0.       -1.5625   -1.325     4.6875    0.      ]
 [ 0.        0.        0.        0.       -1.5625   -0.525     4.6875  ]
 [ 0.        0.        0.        0.        0.078125 -0.3125    3.234375]]
Result: 
 [ -6.94907682  -5.12825434 -15.82874111 -12.48117358  -6.66771239
 -12.39780878  -0.10926383]
True result: 
 [-20.106239999999993, 3.5942399999999997, -0.9676799999999999, 0.0, 0.9676799999999925, -3.5942399999999814, 20.106240000000035]


In [9]:
u = lambda x: x * e ** sin(x)
du = lambda x: e ** sin(x) * (x * cos(x) + 1)
ddu = lambda x: e ** sin(x) * (-x * sin(x) + x * cos(x) ** 2 + 2 * cos(x))

p = lambda x: 1
q = lambda x: -x
r = lambda x: 1
f = lambda x: ddu(x) + q(x) * du(x) - r(x) * u(x)

a = -2
b = 6

alpha_1 = 1
alpha_2 = 0
alpha = -2 * e ** sin(-2)

left_boundary_value = (alpha_1, alpha_2, alpha)

beta_1 = 1
beta_2 = 0
beta = 6 * e ** sin(6)

right_boundary_value = (beta_1, beta_2, beta)

grid_size = 60
grid = np.linspace(a, b, grid_size, retstep=True)
result = solve_equation(p, q, r, f, grid, left_boundary_value, right_boundary_value)
print("Result: \n", result)

points, step = grid
true_solution = []
for point in points:
    true_solution.append(u(point))

print("True result: \n", true_solution)
true_error = get_true_error_norm(true_solution, result)
print("Error: ", true_error)

Matrix: 
 [[   1.          0.          0.       ...    0.          0.
     0.      ]
 [  47.515625 -109.78125    61.265625 ...    0.          0.
     0.      ]
 [   0.         48.015625 -109.78125  ...    0.          0.
     0.      ]
 ...
 [   0.          0.          0.       ... -109.78125    33.265625
     0.      ]
 [   0.          0.          0.       ...   76.015625 -109.78125
    32.765625]
 [   0.          0.          0.       ...    0.          0.
     1.      ]]
Result: 
 [-0.80561425 -0.71510023 -0.64268946 -0.58462764 -0.53778779 -0.49948608
 -0.46730561 -0.43892402 -0.41194003 -0.38369586 -0.35109579 -0.31042812
 -0.25720708 -0.18606425 -0.09073358  0.03581305  0.20101625  0.41211078
  0.67521465  0.99416038  1.3691867   1.79570179  2.26340399  2.75606592
  3.25222355  3.72685333  4.15389471  4.50924278  4.77367355  4.93513661
  4.98997657  4.94289087  4.80571966  4.59540293  4.33156812  4.03420653
  3.72178719  3.40999361  3.11110817  2.83395007  2.58420963  2.36501149
  

In [8]:
def solve_equation_with_accuracy(
        a: float,
        b: float,
        q: Function,
        r: Function,
        f: Function,
        left_boundary_value: BoundaryValue,
        right_boundary_value: BoundaryValue,
        epsilon: float) -> dict:
    grid_size = 4
    grid = np.linspace(a, b, grid_size, retstep=True)
    next_solution = solve_equation(q, r, f, grid, left_boundary_value, right_boundary_value)
    error = float("inf")
    grid_sizes = []
    errors = []
    while error >= epsilon:
        current_solution = next_solution
        grid_size *= 2
        grid = np.linspace(a, b, grid_size, retstep=True)
        next_solution = solve_equation(q, r, f, grid, left_boundary_value, right_boundary_value) 
        error =  get_richardson_error_norm(current_solution, next_solution)
        grid_sizes.append(grid_size)
        errors.append(error)
    clarified_solution = get_richardson_clarified_solution(current_solution, next_solution)
    
    result = {}
    result["clarified_solution"] = clarified_solution
    result["solution"] = next_solution
    result["grid"] = grid
    result["grid_sizes"] = grid_sizes
    result["errors"] = errors
    
    return result 

In [9]:
def run_test(q, r, f, a, b, left_boundary_value, right_boundary_value, epsilon, u=None):
    result = solve_equation_with_accuracy(a, b, q, r, f, left_boundary_value, right_boundary_value, epsilon)
    points, step = result["grid"]
    
    if u is not None:
        true_solution = []
        for point in points:
            true_solution.append(u(point))
        true_error = get_true_error_norm(true_solution, result["solution"])

        print("Real Error: ", true_error)
    
    print("Last Error: ", result["errors"][-1])
    print("Last Grid Size: ", len(points))
    print("Last Step: ", step)

    plt.figure(figsize=(6, 3))
    plt.loglog(result["grid_sizes"], result["errors"], label='Errors', color='red')
    plt.xlabel('grid size')
    plt.ylabel('error norm')
    plt.grid()
    plt.title('Errors')
    plt.show()

### Test 1
#### Simple conditions

$ u = xe^{sin(x)} $ </br>
$ u(-2) = -2e^{sin(-2)} $ </br>
$ u(6) = 6e^{sin(6)} $

$ \varepsilon = 10^{-6} $

In [10]:
u = lambda x: x * e ** sin(x)
du = lambda x: e ** sin(x) * (x * cos(x) + 1)
ddu = lambda x: e ** sin(x) * (-x * sin(x) + x * cos(x) ** 2 + 2 * cos(x))

q = lambda x: -x
r = lambda x: 1
f = lambda x: ddu(x) + q(x) * du(x) - r(x) * u(x)

a = -2
b = 6

alpha_1 = 1
alpha_2 = 0
alpha = -2 * e ** sin(-2)

left_boundary_value = (alpha_1, alpha_2, alpha)

beta_1 = 1
beta_2 = 0
beta = 6 * e ** sin(6)

right_boundary_value = (beta_1, beta_2, beta)

epsilon = 1 / 1000000

run_test(q, r, f, a, b, left_boundary_value, right_boundary_value, epsilon, u)


TypeError: solve_tridiagonal_system() missing 2 required positional arguments: 'c' and 'd'

### Test 2
#### General conditions

$ u(x) = x^{5} - 5x^{4} + 5x^{3} + 5x^{2} - 6x $ </br>
$ 2u(-1) + 3u'(-1) = -24 $ </br>
$ 3u(3) + \frac{3}{8}u'(3) = 3 $

In [ ]:
u = lambda x: x ** 5 - 5 * x ** 4 + 5 * x ** 3 + 5 * x ** 2 - 6 * x
du = lambda x: 5 * x ** 4 - 20 * x ** 3 + 15 * x ** 2 + 10 * x - 6
ddu = lambda x: 10 * (2 * x ** 3 - 6 * x ** 2 + 3 * x + 1)

q = lambda x: 5
r = lambda x: -x
f = lambda x: ddu(x) + q(x) * du(x) - r(x) * u(x)

a = -1
b = 3

alpha_1 = 2
alpha_2 = 1
alpha = -24

left_boundary_value = (alpha_1, alpha_2, alpha)

beta_1 = 3
beta_2 = 1 / 8
beta = 3

right_boundary_value = (beta_1, beta_2, beta)

epsilon = 1 / 1000000

run_test(q, r, f, a, b, left_boundary_value, right_boundary_value, epsilon, u)
# grid = np.linspace(a, b, 8, retstep=True)
# result = solve_equation(q, r, f, grid, left_boundary_value, right_boundary_value)
# print(result)

KeyboardInterrupt: 